### 드레스 피팅 대화 데이터셋을 바탕으로 키워드 추출
- 키워드 추출이 의미가 있을까?
- 그냥 바로 이미지 제작을 위한 프롬프트 생성 해버리는게 나을 것 같아서 그렇게 진행

In [1]:
dialogue = """
<드레스 착용 전>
실장 : 신부님 피부 톤이 살짝 있으셔서 완전 화이트보다는 살짝 아이보리 톤이 들어가고 광택감이 있는 미카도 실크가 어울리실 것 같아요. 혹시 드레스 선택하실 때 생각했던 디자인 있으실까요?
신부 : 제가 팔뚝 살이 좀 있어서 팔을 좀 가리는 디자인이면 좋겠어요.
실장 : 슬리브 있는 드레스는 오히려 팔을 드러낼 수 있어서 오프숄더 디자인이 오히려 팔이 얇아보이실거에요. 오프숄더 디자인으로 한번 준비해드릴게요.
<드레스 착용 후>
실장 : 너무 잘 어울리세요. 베일도 투 베일로 해서 베일로 시선 집중을 시켜서 팔이 더 얇아보이는 효과가 있어요. 신부님이 키가 있으셔서 살짝 슬림한 A라인으로 떨어지는 디자인이 날씬하게 보이는 효과가 있어요. 혹시 티아라는 하실건가요?
신부 : 아뇨, 티아라는 괜찮아요.
실장 : 맞아요. 이 드레스가 작은 비즈가 촘촘히 박혀있어서, 아래쪽이 화려하다보니 티아라까지 하면 좀 투머치일 수 있어요. 식장은 어디쪽으로 잡으셨어요?
신부 : 빌라드지디 청담으로 잡았어요.
실장 : 빌라드지디가 밝은 홀이라 레이스 없이 깔끔한 이런 디자인이 잘 어울려요. 식장이랑도 잘 어울리겠네요.
"""


In [11]:
from openai import OpenAI # openai==1.2.0
import os

client = OpenAI(
    api_key=os.getenv('UPSTAGE_API_KEY'),
    base_url="https://api.upstage.ai/v1/solar"
)
 

prompt = """
  You are a helpful prompt creator.
  Please describe the dress in your user's content in as much detail as possible. 
  This should be in the form of an AI prompt.
  Write in ENGLISH.

  Tips for Best Prompts for AI Art:
  Here are some AI prompt ideas and tips for creating the good AI prompts for realistic art.

  - Use words that the generator can understand easily. Avoid using uncommon or difficult words that the neural networks might not recognize.
  - Make sure to keep your AI art prompts brief and to the point. Avoid giving too much detail, as it can overwhelm the system.
  - Use lots of descriptive words to talk about the subject and layout of your art.
  - Don’t use words that mean the opposite of each other together. It can confuse the AI-generated art style.

  EXAMPLE : street style photo, full shot of a Asian female model wearing an elegant Jamie Bridal gown that emphasizes a pure feminine style, in a lush garden setting, natural lighting, shot on Sony A7S III high resolution digital camera, global illumination

  CONTENT :
"""
stream = client.chat.completions.create(
    model="solar-1-mini-chat",
    messages=[
      {
        "role": "system",
        "content": prompt
      },
      {
        "role": "user",
        "content": dialogue
      }
    ],
    stream=True,
)

result = ''
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        text = chunk.choices[0].delta.content
        print(text, end="")
        result += text

# print(result)
# Use with stream=False
# print(stream.choices[0].message.content)

AI Art Prompt:

Create a detailed illustration of a bride wearing an elegant Jamie Bridal gown made of off-shoulder, sleeveless, and slightly slimmer A-line silhouette with a pure feminine style. The dress should be made of a luxurious and opulent Mikado silk fabric with a slight ivory tone and a subtle sheen. The dress should have a high-quality, well-crafted look, with intricate details such as delicate beading and embroidery. The bride should have a natural, glowing makeup look, with her hair styled in a sophisticated updo. The background should be a lush garden setting with vibrant greenery and flowers, with a soft, natural lighting effect. The overall mood of the illustration should be romantic, elegant, and timeless.

### 위에서 만들어진 프롬프트를 기반으로 이미지를 생성하라고 시킴
- 사용한 모델 : flux
- https://fal.ai/models/fal-ai/flux/dev/examples

In [ ]:
os.environ['FAL_KEY'] = os.getenv('FAL_KEY')

In [27]:
import fal_client

handler = fal_client.submit(
    "fal-ai/flux/dev",
    arguments={
        "prompt": prompt,
        "image_size": "landscape_4_3",
        "num_inference_steps": 28,
        "guidance_scale": 3.5,
        "num_images": 1,
        "enable_safety_checker": True
    },
)

result = handler.get()
print(result)

{'images': [{'url': 'https://fal.media/files/koala/tcZrWYS0bIq7nMVz6YGn7.png', 'width': 1024, 'height': 768, 'content_type': 'image/jpeg'}], 'timings': {'inference': 2.168901782017201}, 'seed': 636616846, 'has_nsfw_concepts': [False], 'prompt': 'create a detailed illustration of a bride wearing an elegant jamie bridal gown made of off-shoulder, sleeveless, and slightly slimmer a-line silhouette with a pure feminine style. the dress should be made of a luxurious and opulent mikado silk fabric with a slight ivory tone and a subtle sheen. the dress should have a high-quality, well-crafted look, with intricate details such as delicate beading and embroidery. the bride should have a natural, glowing makeup look, with her hair styled in a sophisticated updo. the background should be a lush garden setting with vibrant greenery and flowers, with a soft, natural lighting effect. the overall mood of the illustration should be romantic, elegant, and timeless.'}


### 결과

In [30]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://fal.media/files/koala/tcZrWYS0bIq7nMVz6YGn7.png")

### 정리
- 문제 
1. 프롬프트를 일단 대충 만들어서 형식이 바뀔 경우가 있다
2. 드레스 자체를 hallucination 하는 경우가 다수 존재하는 것으로 파악된다.